In [4]:

import os

from collections import defaultdict
label_dir = 'valid/labels'
class_counts = defaultdict(int)

for filename in os.listdir(label_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(label_dir, filename), 'r') as f:
            for line in f:
                if line.strip():
                    class_id = line.strip().split()[0]
                    class_counts[class_id] += 1

print("\nInstances per class:")
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count}")


Instances per class:
Class 0: 252
Class 1: 243


In [6]:

import os

from collections import defaultdict
label_dir = 'train/labels'
class_counts = defaultdict(int)

for filename in os.listdir(label_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(label_dir, filename), 'r') as f:
            for line in f:
                if line.strip():
                    class_id = line.strip().split()[0]
                    class_counts[class_id] += 1

print("\nInstances per class:")
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count}")


Instances per class:
Class 1: 789
Class 0: 717


In [36]:
#this code is to balance the dataset (with help of chatgpt)
import os
import random

base_dirs = ['train', 'valid']
class_ids = [0, 1]  
image_exts = ['.jpg', '.jpeg', '.png']

def count_instances_per_image(label_path):
    # Count instances of each class in a single label file
    counts = {0:0, 1:0}
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 1:
                continue
            cls = int(parts[0])
            if cls in counts:
                counts[cls] += 1
    return counts

def remove_images(files_to_remove, images_dir, labels_dir):
    for fname in files_to_remove:
        label_path = os.path.join(labels_dir, fname + '.txt')
        image_path = None
        for ext in image_exts:
            img_path = os.path.join(images_dir, fname + ext)
            if os.path.exists(img_path):
                image_path = img_path
                break

        if os.path.exists(label_path):
            os.remove(label_path)
        if image_path and os.path.exists(image_path):
            os.remove(image_path)

def balance_dataset(folder):
    images_dir = os.path.join(folder, 'images')
    labels_dir = os.path.join(folder, 'labels')

    all_labels = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

    # Count total instances per class
    total_counts = {0:0, 1:0}
    per_image_counts = {}

    for label_file in all_labels:
        label_path = os.path.join(labels_dir, label_file)
        counts = count_instances_per_image(label_path)
        per_image_counts[label_file[:-4]] = counts
        total_counts[0] += counts[0]
        total_counts[1] += counts[1]

    print(f"{folder} total instances before balancing: {total_counts}")

    # Determine majority and minority class
    if total_counts[0] > total_counts[1]:
        majority_class = 0
        minority_count = total_counts[1]
    else:
        majority_class = 1
        minority_count = total_counts[0]

    # Current majority class instances to remove to balance
    excess = total_counts[majority_class] - minority_count
    if excess <= 0:
        print(f"{folder} already balanced.")
        return

    print(f"{folder} majority class: {majority_class} with {total_counts[majority_class]} instances. Need to remove {excess} instances.")

    # List images containing majority class
    images_with_majority = [img for img, counts in per_image_counts.items() if counts[majority_class] > 0]

    # Shuffle so random removal
    random.shuffle(images_with_majority)

    removed_instances = 0
    removed_images = []

    for img in images_with_majority:
        if removed_instances >= excess:
            break
        count_in_image = per_image_counts[img][majority_class]
        removed_instances += count_in_image
        removed_images.append(img)

    print(f"Removing {len(removed_images)} images to reduce {removed_instances} instances.")

    # Remove selected images and labels
    remove_images(removed_images, images_dir, labels_dir)

    # Final counts after removal
    total_counts_after = {0:0, 1:0}
    for label_file in os.listdir(labels_dir):
        if label_file.endswith('.txt'):
            c = count_instances_per_image(os.path.join(labels_dir, label_file))
            total_counts_after[0] += c[0]
            total_counts_after[1] += c[1]

    print(f"{folder} total instances after balancing: {total_counts_after}")

for folder in base_dirs:
    balance_dataset(folder)


train total instances before balancing: {0: 789, 1: 1366}
train majority class: 1 with 1366 instances. Need to remove 577 instances.
Removing 404 images to reduce 577 instances.
train total instances after balancing: {0: 717, 1: 789}
valid total instances before balancing: {0: 386, 1: 252}
valid majority class: 0 with 386 instances. Need to remove 134 instances.
Removing 92 images to reduce 134 instances.
valid total instances after balancing: {0: 252, 1: 243}


In [ ]:
!pip install ultralytics
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  

model.train(
    data='data.yaml',     
    epochs=50,            
    imgsz=512,             
    batch=4,               # Lower for M4 chip to avoid OOM issues
    device='cpu',          # Uses Apple CPU
    workers=0,             # Prevents macOS DataLoader multiprocessing issues
    optimizer='Adam',      # More stable than SGD for small batches
    name='Training1',
    verbose=True  # show detailed metrics per epoch
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 3.0 MB/s eta 0:00:0000:0100:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/nitingupta/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████████████████████████████████| 6.25M/6.25M [00:02<00:00, 2.86MB/s]


Ultralytics 8.3.158 🚀 Python-3.12.7 torch-2.7.0 CPU (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Training1, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, r

train: Scanning /Users/nitingupta/Desktop/training/VOC1.v1i.yolov8/train/labels.

train: New cache created: /Users/nitingupta/Desktop/training/VOC1.v1i.yolov8/train/labels.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 198.8±236.5 MB/s, size: 30.8 KB)


...

                   all        338        495       0.86      0.803       0.88      0.722

50 epochs completed in 2.347 hours.


Optimizer stripped from runs/detect/Training1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/Training1/weights/best.pt, 6.2MB

Validating runs/detect/Training1/weights/best.pt...
Ultralytics 8.3.158 🚀 Python-3.12.7 torch-2.7.0 CPU (Apple M4)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        338        495      0.861      0.803       0.88      0.722
                   car        195        252      0.911      0.853       0.92      0.786
                people        161        243       0.81      0.753       0.84      0.659
Speed: 0.4ms preprocess, 29.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/Training1


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x3291f7bc0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [12]:


!pip install ultralytics
from ultralytics import YOLO

model = YOLO('runs/detect/Training1/weights/last.pt')  

model.train(
    data='data.yaml',      
    epochs=50,             
    imgsz=512,             # Better resolution for COCO
    batch=4,               # Lower for M3 chip to avoid OOM issues
    device='cpu',          # Uses Apple CPU
    workers=0,             # Prevents macOS DataLoader multiprocessing issues 
    optimizer='Adam',      # More stable than SGD for small batches
    name='Training2',
    verbose=True  # show detailed metrics per epoch
)

Ultralytics 8.3.158 🚀 Python-3.12.7 torch-2.7.0 CPU (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/Training1/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Training2, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pro

train: Scanning /Users/nitingupta/Desktop/training/VOC1.v1i.yolov8/train/labels.

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 178.9±207.4 MB/s, size: 30.8 KB)



val: Scanning /Users/nitingupta/Desktop/training/VOC1.v1i.yolov8/valid/labels.ca

Plotting labels to runs/detect/Training2/labels.jpg... 


...

                   all        338        495      0.869      0.827      0.885      0.739

50 epochs completed in 2.562 hours.


Optimizer stripped from runs/detect/Training2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/Training2/weights/best.pt, 6.2MB

Validating runs/detect/Training2/weights/best.pt...
Ultralytics 8.3.158 🚀 Python-3.12.7 torch-2.7.0 CPU (Apple M4)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        338        495      0.869      0.828      0.885      0.739
                   car        195        252      0.905      0.873      0.922      0.796
                people        161        243      0.833      0.782      0.847      0.682
Speed: 0.4ms preprocess, 28.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/Training2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x329b51eb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [13]:
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import os


#  Model and Paths

model_path = "runs/detect/Training2/weights/best.pt"  
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Path not found: {model_path}")

# Load the YOLO model
model = YOLO(model_path)


# Evaluate the Model

results = model.val()

#  Extract Metrics

precision = results.box.p           # Numpy array of precision per class
recall = results.box.r              # Numpy array of recall per class
ap50 = results.box.ap50            # Numpy array of mAP@0.5 per class
class_names = list(results.names.values())


#  Plot the Metrics

x = np.arange(len(class_names))

plt.figure(figsize=(10, 6))
plt.bar(x - 0.2, precision, 0.2, label='Precision', color='steelblue')
plt.bar(x, recall, 0.2, label='Recall', color='orange')
plt.bar(x + 0.2, ap50, 0.2, label='AP@0.5', color='green')

# Labels & Styling
plt.xticks(x, class_names, rotation=45, fontsize=12)
plt.ylabel("Score", fontsize=12)
plt.xlabel("Class", fontsize=12)
plt.title("YOLOv8 Per-Class Metrics", fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, axis='y', linestyle='--', alpha=0.7)

# Adjust layout and save
plt.tight_layout()
plt.savefig("runs/detect/Training2/yolo_metrics.png", dpi=300)  # Export the plot
plt.show()

#  Print Summary

print("\n=== YOLOv8 Evaluation Summary ===")
for cls_name, p, r, a in zip(class_names, precision, recall, ap50):
    print(f"Class: {cls_name:>10} | Precision: {p:.3f} | Recall: {r:.3f} | mAP@0.5: {a:.3f}")

all_metrics = {
    "precision": results.box.p,
    "recall": results.box.r,
    "mAP@0.5": results.box.ap50,
}
print("\n Done! Metrics saved to 'yolo_metrics.png'.")

Ultralytics 8.3.158 🚀 Python-3.12.7 torch-2.7.0 CPU (Apple M4)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 138.4±103.8 MB/s, size: 30.4 KB)


val: Scanning /Users/nitingupta/Desktop/training/VOC1.v1i.yolov8/valid/labels.ca
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        338        495      0.871      0.826      0.885      0.739
                   car        195        252      0.905      0.873      0.922      0.795
                people        161        243      0.837      0.778      0.848      0.684
Speed: 0.2ms preprocess, 55.8ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/val6


<Figure size 1000x600 with 1 Axes>


=== YOLOv8 Evaluation Summary ===
Class:        car | Precision: 0.905 | Recall: 0.873 | mAP@0.5: 0.922
Class:     people | Precision: 0.837 | Recall: 0.778 | mAP@0.5: 0.848

✅ Done! Metrics saved to 'yolo_metrics.png'.
